In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

<h1>10 Data Aggregation and Group Operations</h1>

<p>Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, can be a critical component of a data analysis workflow. After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly pivot tables for reporting or visualization purposes. pandas provides a versatile <code>groupby</code> interface, enabling you to slice, dice, and summarize datasets in a natural way.</p>

<p>One reason for the popularity of relational databases and SQL (which stands for “structured query language”) is the ease with which data can be joined, filtered, transformed, and aggregated. However, query languages like SQL impose certain limitations on the kinds of group operations that can be performed. As you will see, with the expressiveness of Python and pandas, we can perform quite complex group operations by expressing them as custom Python functions that manipulate the data associated with each group. In this chapter, you will learn how to:</p>

<ul>
    <li>Split a pandas object into pieces using one or more keys (in the form of functions, arrays, or DataFrame column names)</li>
    <li>Calculate group summary statistics, like count, mean, or standard deviation, or a user-defined function</li>
    <li>Apply within-group transformations or other manipulations, like normalization, linear regression, rank, or subset selection</li>
    <li>Compute pivot tables and cross-tabulations</li>
    <li>Perform quantile analysis and other statistical group analyses</li>
</ul>

<p><strong>Note</strong><br />
Time-based aggregation of time series data, a special use case of <code>groupby</code>, is referred to as <em>resampling</em> in this book and will receive separate treatment in Ch 11: Time Series.</p>

<p>As with the rest of the chapters, we start by importing NumPy and pandas:</p>

In [2]:
import numpy as np
import pandas as pd

<h2>How to Think About Group Operations</h2>
<p>Hadley Wickham, an author of many popular packages for the R programming language, coined the term <em>split-apply-combine</em> for describing group operations. In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is <em>split</em> into groups based on one or more <em>keys</em> that you provide. The splitting is performed on a particular axis of an object. For example, a DataFrame can be grouped on its rows (<code>axis="index"</code>) or its columns (<code>axis="columns"</code>). Once this is done, a function is applied to each group, producing a new value. Finally, the results of all those function applications are combined into a result object. The form of the resulting object will usually depend on what’s being done to the data. See next figure for a mockup of a simple group aggregation.<img src='images/pda3_1001.png' alt='Illustration of a group aggregation'></p>
<p>Each grouping key can take many forms, and the keys do not have to be all of the same type:</p>
<ul>
    <li>A list or array of values that is the same length as the axis being grouped</li>
    <li>A value indicating a column name in a DataFrame</li>
    <li>A dictionary or Series giving a correspondence between the values on the axis being grouped and the group names</li>
    <li>A function to be invoked on the axis index or the individual labels in the index</li>
</ul>
<p>Note that the latter three methods are shortcuts for producing an array of values to be used to split up the object. Don’t worry if this all seems abstract. Throughout this chapter, I will give many examples of all these methods. To get started, here is a small tabular dataset as a DataFrame:</p>

In [3]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

<p>Suppose you wanted to compute the mean of the <code>data1</code> column using the labels from <code>key1</code>. There are a number of ways to do this. One is to access <code>data1</code> and call <code>groupby</code> with the column (a Series) at <code>key1</code>:</p>

In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

<p>This <code>grouped</code> variable is now a special "GroupBy" object. It has not actually computed anything yet except for some intermediate data about the group key <code>df["key1"]</code>. The idea is that this object has all of the information needed to then apply some operation to each of the groups. For example, to compute group means we can call the GroupBy’s <code>mean</code> method:</p>

In [5]:
grouped.mean()

<p>Later in <a href="#data_aggregation">Data Aggregation</a>, I'll explain more about what happens when you call <code>.mean()</code>. The important thing here is that the data (a Series) has been aggregated by splitting the data on the group key, producing a new Series that is now indexed by the unique values in the <code>key1</code> column. The result index has the name <code>"key1"</code> because the DataFrame column <code>df["key1"]</code> did.</p>

<p>If instead we had passed multiple arrays as a list, we'd get something different:</p>

In [6]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

<p>Here we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed:</p>

In [7]:
means.unstack()

<p>In this example, the group keys are all Series, though they could be any arrays of the right length:</p>

In [8]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

<p>Frequently, the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names (whether those are strings, numbers, or other Python objects) as the group keys:</p>

In [9]:
df.groupby("key1").mean()

In [10]:
df.groupby("key2").mean()

In [11]:
df.groupby(["key1", "key2"]).mean()

<p>You may have noticed in the second case, <code>df.groupby("key2").mean()</code>, that there is no <code>key1</code> column in the result. Because <code>df["key1"]</code> is not numeric data, it is said to be a nuisance column, which is therefore automatically excluded from the result. By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset, as you’ll see soon.</p>

<p>Regardless of the objective in using groupby, a generally useful <code>GroupBy</code> method is <code>size</code>, which returns a Series containing group sizes:</p>

In [12]:
df.groupby(["key1", "key2"]).size()

<p>Note that any missing values in a group key are excluded from the result by default. This behavior can be disabled by passing <code>dropna=False</code> to <code>groupby</code>:</p>

In [13]:
df.groupby("key1", dropna=False).size()

In [14]:
df.groupby(["key1", "key2"], dropna=False).size()

<p>A group function similar in spirit to <code>size</code> is <code>count</code>, which computes the number of nonnull values in each group:</p>

In [15]:
df.groupby("key1").count()

<h2>Iterating over Groups</h2>

<p>The object returned by <code>groupby</code> supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. Consider the following:</p>

In [16]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

<p>In the case of multiple keys, the first element in the tuple will be a tuple of key values:</p>

In [17]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)


<p>Of course, you can choose to do whatever you want with the pieces of data. A recipe you may find useful is computing a dictionary of the data pieces as a one-liner:</p>

In [18]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

<p>By default <code>groupby</code> groups on <code>axis="index"</code>, but you can group on any of the other axes. For example, we could group the columns of our example <code>df</code> here by whether they start with <code>"key"</code> or <code>"data"</code>:</p>

In [19]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, 
                     axis="columns")

<p>We can print out the groups like so:</p>

In [20]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

<h2>Selecting a Column or Subset of Columns</h2>

<p>Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:</p>
<pre>df.groupby('key1')['data1']
df.groupby('key1')[['data2']]</pre>
<p>are conveniences for:</p>
<pre>df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])</pre>
<p>Especially for large datasets, it may be desirable to aggregate only a few columns. For example, in the preceding dataset, to compute the means for just the <code>data2</code> column and get the result as a DataFrame, we could write:</p>

In [21]:
df.groupby(["key1", "key2"])[["data2"]].mean()

<p>The object returned by this indexing operation is a grouped DataFrame if a list or array is passed, or a grouped Series if only a single column name is passed as a scalar:</p>

In [22]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [23]:
s_grouped.mean()

<h2>Grouping with Dictionaries and Series</h2>

<p>Grouping information may exist in a form other than an array. Let’s consider another example DataFrame:</p>

In [24]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

<p>Now, suppose I have a group correspondence for the columns and want to sum the columns by group:</p>

In [25]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

<p>Now, you could construct an array from this dictionary to pass to <code>groupby</code>, but instead we can just pass the dictionary (I included the key <code>"f"</code> to highlight that unused grouping keys are OK):</p>

In [26]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

<p>The same functionality holds for Series, which can be viewed as a fixed-size mapping:</p>

In [27]:
map_series = pd.Series(mapping)
map_series

In [28]:
people.groupby(map_series, axis="columns").count()

<h2>Grouping with Functions</h2>

<p>Using Python functions is a more generic way of defining a group mapping compared with a dictionary or Series. Any function passed as a group key will be called once per index value (or once per column value if using <code>axis="columns"</code>), with the return values being used as the group names. More concretely, consider the example DataFrame from the previous section, which has people’s first names as index values. Suppose you wanted to group by name length. While you could compute an array of string lengths, it's simpler to just pass the <code>len</code> function:</p>

In [29]:
people.groupby(len).sum()

<p>Mixing functions with arrays, dictionaries, or Series is not a problem, as everything gets converted to arrays internally:</p>

In [30]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

<h2>Grouping by Index Levels</h2>

<p>A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index. Let's look at an example:</p>

In [31]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)),
                       columns=columns)
hier_df

<p>To group by level, pass the level number or name using the <code>level</code> keyword:</p>

In [32]:
hier_df.groupby(level="cty", axis="columns").max()

<span id="data_aggregation"></span><h2>Data Aggregation</h2>

<p><em>Aggregations</em> refer to any data transformation that produces scalar values from arrays. The preceding examples have used several of them, including <code>mean</code>, <code>count</code>, <code>min</code>, and <code>sum</code>. You may wonder what is going on when you invoke <code>mean()</code> on a GroupBy object. Many common aggregations, such as those found in <a href="#table_10.1">Table 10.1</a>, have optimized implementations. However, you are not limited to only this set of methods.</p>
<table id="table_10.1">
    <caption>Table 10.1: Optimized <code>groupby</code> methods</caption>
    <tr>
        <th>Function name</th>
        <th>Descripction</th>
    </tr>
    <tr>
        <td><code>any, all</code></td>
        <td>Return <code>True</code> if any (one or more values) or all non-NA values are "truthy"</td>
    </tr>
    <tr>
        <td><code>count</code></td>
        <td>Number of non-NA values</td>
    </tr>
    <tr>
        <td><code>cummin, cummax</code></td>
        <td>Cumulative minimum and maximum of non-NA values</td>
    </tr>
    <tr>
        <td><code>cumsum</code></td>
        <td>Cumulative sum of non-NA values</td>
    </tr>
    <tr>
        <td><code>cumprod</code></td>
        <td>Cumulative product of non-NA values</td>
    </tr>
    <tr>
        <td><code>first, last</code></td>
        <td>Mean of non-NA valuesFirst and last non-NA values</td>
    </tr>
    <tr>
        <td><code>median</code></td>
        <td>Arithmetic median of non-NA values</td>
    </tr>
    <tr>
        <td><code>min, max</code></td>
        <td>Minimum and maximum of non-NA values</td>
    </tr>
    <tr>
        <td><code>nth</code></td>
        <td>Retrieve value that would appear at position <code>n</code> with the data in sorted order</td>
    </tr>
    <tr>
        <td><code>ohlc</code></td>
        <td>Compute four "open-high-low-close" statistics for time series-like data</td>
    </tr>
    <tr>
        <td><code>prod</code></td>
        <td>Product of non-NA values</td>
    </tr>
    <tr>
        <td><code>quantile</code></td>
        <td>Compute sample quantile</td>
    </tr>
    <tr>
        <td><code>rank</code></td>
        <td>Ordinal ranks of non-NA values, like calling <code>Series.rank</code></td>
    </tr>
    <tr>
        <td><code>size</code></td>
        <td>Compute group sizes, returning results as a Series</td>
    </tr>
    <tr>
        <td><code>sum</code></td>
        <td>Sum of non-NA values</td>
    </tr>
    <tr>
        <td><code>std, var</code></td>
        <td>Sample standard deviation and variance</td>
    </tr>
</table>
<p>You can use aggregations of your own devising and additionally call any method that is also defined on the object being grouped. For example, the <code>nsmallest</code> Series method selects the smallest requested number of values from the data. While <code>nsmallest</code> is not explicitly implemented for GroupBy, we can still use it with a nonoptimized implementation. Internally, GroupBy slices up the Series, calls <code>piece.nsmallest(n)</code> for each piece, and then assembles those results into the result object:</p>   

In [33]:
df

In [34]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

<p>To use your own aggregation functions, pass any function that aggregates an array to the <code>aggregate</code> method or its short alias <code>agg</code>:</p>

In [35]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

<p>You may notice that some methods, like <code>describe</code>, also work, even though they are not aggregations, strictly speaking:</p>

In [36]:
grouped.describe()

<p>I will explain in more detail what has happened here in <a href="#apply">Apply: General split-apply-combine</a>.</p>

<p><strong>Note</strong><br>
Custom aggregation functions are generally much slower than the optimized functions found in <a href="#table_10.1">Table 10.1</a>. This is because there is some extra overhead (function calls, data rearrangement) in constructing the intermediate group data chunks.</p>

<h3>Column-Wise and Multiple Function Application</h3>

<p>Let's return to the tipping dataset used in the last chapter. After loading it with <code>pandas.read_csv</code>, we add a tipping percentage column:</p>

In [37]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

<p>Now I will add a <code>tip_pct</code> column with the tip percentage of the total bill:</p>

In [38]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

<p>As you’ve already seen, aggregating a Series or all of the columns of a DataFrame is a matter of using <code>aggregate</code> (or <code>agg</code>) with the desired function or calling a method like <code>mean</code> or <code>std</code>. However, you may want to aggregate using a different function, depending on the column, or multiple functions at once. Fortunately, this is possible to do, which I’ll illustrate through a number of examples. First, I’ll group the <code>tips</code> by <code>day</code> and <code>smoker</code>:</p>

In [39]:
grouped = tips.groupby(["day", "smoker"])

<p>Note that for descriptive statistics like those in <a href="#table_10.1">Table 10.1</a>, you can pass the name of the function as a string:</p>

In [40]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

<p>If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions:</p>

In [41]:
grouped_pct.agg(["mean", "std", peak_to_peak])

<p>Here we passed a list of aggregation functions to <code>agg</code> to evaluate independently on the data groups.</p>

<p>You don’t need to accept the names that GroupBy gives to the columns; notably, <code>lambda</code> functions have the name <code>"&lt;lambda&gt;"</code>, which makes them hard to identify (you can see for yourself by looking at a function’s <code>__name__</code> attribute). Thus, if you pass a list of <code>(name, function)</code> tuples, the first element of each tuple will be used as the DataFrame column names (you can think of a list of 2-tuples as an ordered mapping):</p>

In [42]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

<p>With a DataFrame you have more options, as you can specify a list of functions to apply to all of the columns or different functions per column. To start, suppose we wanted to compute the same three statistics for the <code>tip_pct</code> and <code>total_bill</code> columns:</p>

In [43]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

<p>As you can see, the resulting DataFrame has hierarchical columns, the same as you would get aggregating each column separately and using <code>concat</code> to glue the results together using the column names as the <code>keys</code> argument:</p>

In [44]:
result["tip_pct"]

<p>As before, a list of tuples with custom names can be passed:</p>

In [45]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

<p>Now, suppose you wanted to apply potentially different functions to one or more of the columns. To do this, pass a dictionary to <code>agg</code> that contains a mapping of column names to any of the function specifications listed so far:</p>

In [46]:
grouped.agg({"tip" : np.max, "size" : "sum"})

In [47]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

<p>A DataFrame will have hierarchical columns only if multiple functions are applied to at least one column.</p>

<h3>Returning Aggregated Data Without Row Indexes</h3>

<p>In all of the examples up until now, the aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations. Since this isn’t always desirable, you can disable this behavior in most cases by passing <code>as_index=False</code> to <code>groupby</code>:</p>

In [48]:
tips.groupby(["day", "smoker"], as_index=False).mean()

<p>Of course, it’s always possible to obtain the result in this format by calling <code>reset_index</code> on the result. Using the <code>as_index=False</code> argument avoids some unnecessary computations.</p>

<span id="apply"></span><h2>10.3 Apply: General split-apply-combine</h2>

<p id="apply">The most general-purpose GroupBy method is <code>apply</code>, which is the subject of this section. <code>apply</code> splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces.</p>

<p>Returning to the tipping dataset from before, suppose you wanted to select the top five <code>tip_pct</code> values by group. First, write a function that selects the rows with the largest values in a particular column:</p>

In [49]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

<p>Now, if we group by <code>smoker</code>, say, an call <code>apply</code> with this function, we get the followind:</p>

In [50]:
tips.groupby("smoker").apply(top)

<p>What has happened here? First, the <code>tips</code> DataFrame is split into groups based on the value of <code>smoker</code>. Then the <code>top</code> function is called on each group, and the results of each function call are glued together using <code>pandas.concat</code>, labeling the pieces with the group names. The result therefore has a hierarchical index with an inner level that contains index values from the original DataFrame.</p>

<p>If you pass a function to <code>apply</code> that takes other arguments or keywords, you can pass these after the function:</p>

In [51]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

<p>Beyond these basic usage mechanics, getting the most out of <code>apply</code> may require some creativity. What occurs inside the function passed is up to you; it must either return a pandas object or a scalar value. The rest of this chapter will consist mainly of examples showing you how to solve various problems using <code>groupby</code>.</p>

<p>For example, you may recall that I earlier called <code>describe</code> on a GroupBy object:</p>

In [52]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

In [53]:
result.unstack("smoker")

<p>Inside GroupBy, when you invoke a method like <code>describe</code>, it is actually just a shortcut for:</p>
<pre>
<strong>def</strong> f(group):
    <strong>return</strong> group.describe()
    
grouped.apply(f)
</pre>

<h3>Suppressing the Group Keys</h3>

<p>In the preceding examples, you see that the resulting object has a hierarchical index formed from the group keys, along with the indexes of each piece of the original object. You can disable this by passing <code>group_keys=False</code> to <code>groupby</code>:</p>

In [54]:
tips.groupby("smoker", group_keys=False).apply(top)

<h3>Quantile and Bucket Analysis</h3>

As you may recall from Ch 8: Data Wrangling: Join, Combine, and Reshape, pandas has some tools, in particular <code>pandas.cut</code> and <code>pandas.qcut</code>, for slicing data up into buckets with bins of your choosing, or by sample quantiles. Combining these functions with <code>groupby</code> makes it convenient to perform bucket or quantile analysis on a dataset. Consider a simple random dataset and an equal-length bucket categorization using <code>pandas.cut</code>:</p>

In [55]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

In [56]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

<p>The <code>Categorical</code> object returned by <code>cut</code> can be passed directly to <code>groupby</code>. So we could compute a set of group statistics for the quartiles, like so:</p>

In [57]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

<p>Keep in mind the same result could have been computed more simply with:</p>

In [58]:
grouped.agg(["min", "max", "count", "mean"])

<p>These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use <code>pandas.qcut</code>. We can pass <code>4</code> as the number of bucket compute sample quartiles, and pass <code>labels=False</code> to obtain just the quartile indices instead of intervals:</p>

In [59]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

In [60]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

<h3>Example: Filling Missing Values with Group-Specific Values</h3>

<p>When cleaning up missing data, in some cases you will remove data observations using <code>dropna</code>, but in others you may want to fill in the null (NA) values using a fixed value or some value derived from the data. <code>fillna</code> is the right tool to use; for example, here I fill in the null values with the mean:</p>

In [61]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

In [62]:
s.fillna(s.mean())

<p>Suppose you need the fill value to vary by group. One way to do this is to group the data and use <code>apply</code> with a function that calls <code>fillna</code> on each data chunk. Here is some sample data on US states divided into eastern and western regions:</p>

In [63]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

<p>Let's set some values in the data to be missing:</p>

In [64]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

In [65]:
data.groupby(group_key).size()

In [66]:
data.groupby(group_key).count()

In [67]:
data.groupby(group_key).mean()

<p>We can fill the NA values using the group means, like so:</p>

In [68]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key, group_keys=False).apply(fill_mean)

<p>In another case, you might have predefined fill values in your code that vary by group. Since the groups have a <code>name</code> attribute set internally, we can use that:</p>

In [69]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key, group_keys=False).apply(fill_func)

<h3>Example: Random Sampling and Permutation</h3>

<p>Suppose you wanted to draw a random sample (with or without replacement) from a large dataset for Monte Carlo simulation purposes or some other application. There are a number of ways to perform the “draws”; here we use the <code>sample</code> method for Series.</p>

<p>To demonstrate, here’s a way to construct a deck of English-style playing cards:</p>

In [70]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

<p>Now we have a Series of length 52 whose index contains card names, and values are the ones used in blackjack and other games (to keep things simple, I let the ace "A" be 1):</p>

In [71]:
deck.head(13)

<p>Now, based on what I said before, drawing a hand of five cards from the deck could be written as:</p>

In [72]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

<p>Suppose you wanted two random cards from each suit. Because the suit is the last character of each card name, we can group based on this and use <code>apply</code>:</p>

In [73]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

<p>Alternatively, we could pass <code>group_keys=False</code> to drop the outer suit index, leaving in just the selected cards:</p>

In [74]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

<h3>Example: Group Weighted Average and Correlation</h3>

<p>Under the split-apply-combine paradigm of <code>groupby</code>, operations between columns in a DataFrame or two Series, such as a group weighted average, are possible. As an example, take this dataset containing group keys, values, and some weights:</p>

In [75]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

<p>The weighted average by <code>category</code> would then be:</p>

In [76]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg)

<p>As another example, consider a financial dataset originally obtained from Yahoo! Finance containing end-of-day prices for a few stocks and the S&amp;P 500 index (the SPX symbol):</p>

In [77]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)
close_px.info()

In [78]:
close_px.tail(4)

<p>The DataFrame <code>info()</code> method here is a convenient way to get an overview of the contents of a DataFrame.</p>

<p>One task of interest might be to compute a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with <code>SPX</code>. As one way to do this, we first create a function that computes the pair-wise correlation of each column with the <code>"SPX"</code> column:</p>

In [79]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

<p>Next, we compute percent change on <code>close_px</code> using <code>pct_change</code>:</p>

In [80]:
rets = close_px.pct_change().dropna()

<p>Lastly, we group these percent changes by year, which can be extracted from each row label with a one-line function that returns the <code>year</code> attribute of each <code>datetime</code> label:</p>

In [81]:
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

<p>You could also compute intercolumn correlations. Here we compute the annual correlation between Apple and Microsoft:</p>

In [82]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])
by_year.apply(corr_aapl_msft)

<h3>Example: Group-Wise Linear Regression</h3>

<p>In the same theme as the previous example, you can use <code>groupby</code> to perform more complex group-wise statistical analysis, as long as the function returns a pandas object or scalar value. For example, I can define the following <code>regress</code> function (using the <code>statsmodels</code> econometrics library), which executes an ordinary least squares (OLS) regression on each chunk of data:</p>

In [83]:
import statsmodels.api as sm
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

<p>You can install <code>statsmodels</code> with conda if you don't have it already:</p>

<pre>conda install statsmodels</pre>

<p>Now, to run a yearly linear regression of AAPL on SPX returns, execute:</p>

In [84]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

<h2>10.4 Group Transforms and "Unwrapped" GroupBys</h2>

<p>In <a href="#apply">Apply: General split-apply-combine</a>, we looked at the <code>apply</code> method in grouped operations for performing transformations. There is another built-in method called <code>transform</code>, which is similar to <code>apply</code> but imposes more constraints on the kind of function you can use:</p>
<ul>
    <li>It can produce a scalar value to be broadcast to the shape of the group.</li>

    <li>It can produce an object of the same shape as the input group.</li>

    <li>It must not mutate its input.</li>
</ul>

<p>Let's consider a simple example for illustration:</p>

In [85]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

<p>Here are the group means by key:</p>

In [86]:
g = df.groupby('key', group_keys=False)['value']
g.mean()

<p>Suppose instead we wanted to produce a Series of the same shape as <code>df['value']</code> but with values replaced by the average grouped by <code>'key'</code>. We can pass a function that computes the mean of a single group to transform:</p>

In [87]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

<p>For built-in aggregation functions, we can pass a string alias as with the GroupBy <code>agg</code> method:</p>

In [88]:
g.transform('mean')

<p>Like <code>apply</code>, <code>transform</code> works with functions that return Series, but the result must be the same size as the input. For example, we can multiply each group by 2 using a helper function:</p>

In [89]:
def times_two(group):
    return group * 2
g.transform(times_two)

<p>As a more complicated example, we can compute the ranks in descending order for each group:</p>

In [90]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

<p>Consider a group transformation function composed from simple aggregations:</p>

In [91]:
def normalize(x):
    return (x - x.mean()) / x.std()

<p>We can obtain equivalent results in this case using either <code>transform</code> or <code>apply</code>:</p>

In [92]:
g.transform(normalize)

In [93]:
g.apply(normalize)

<p>Built-in aggregate functions like <code>'mean'</code> or <code>'sum'</code> are often much faster than a general <code>apply</code> function. These also have a "fast path" when used with <code>transform</code>. This allows us to perform what is called an <em>unwrapped</em> group operation:</p>

In [94]:
g.transform('mean')

In [95]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

<p>Here, we are doing arithmetic between the outputs of multiple GroupBy operations instead of writing a function and passing it to <code>groupby(...).apply</code>. That is what is meant by "unwrapped."</p>

<p>While an unwrapped group operation may involve multiple group aggregations, the overall benefit of vectorized operations often outweighs this.</p>

<h2>10.5 Pivot Tables and Cross-Tabulation</h2>

<p>A <em>pivot table</em> is a data summarization tool frequently found in spreadsheet programs and other data analysis software. It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns. Pivot tables in Python with pandas are made possible through the <code>groupby</code> facility described in this chapter, combined with reshape operations utilizing hierarchical indexing. DataFrame also has a <code>pivot_table</code> method, and there is also a top-level <code>pandas.pivot_table</code> function. In addition to providing a convenience interface to <code>groupby</code>, <code>pivot_table</code> can add partial totals, also known as margins.</p>

<p>Returning to the tipping dataset, suppose you wanted to compute a table of group means (the default <code>pivot_table</code> aggregation type) arranged by <code>day</code> and <code>smoker</code> on the rows:</p>

In [96]:
tips.head()

<p>For avoiding next message:</p>
<pre>
FutureWarning: pivot_table dropped a column because
it failed to aggregate. This behavior is deprecated
and will raise in a future version of pandas. Select
only the columns that can be aggregated.
</pre>
<p>use <code>values</code> parameter with <code>pivot_table</code>.</p>

In [97]:
tips.pivot_table(index=["day","smoker"],
                values=['size','tip','tip_pct','total_bill'])

<p>This could have been produced with <code>groupby</code> directly, using <code>tips.groupby(["day", "smoker"]).mean()</code>.</p>

In [98]:
tips.groupby(['day','smoker'])\
             [['size','tip','tip_pct','total_bill']].mean()

<p>Now, suppose we want to take the average of only <code>tip_pct</code> and <code>size</code>, and additionally group by <code>time</code>. I’ll put smoker in the table columns and time and day in the rows:

In [99]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

<p>We could augment this table to include partial totals by passing <code>margins=True</code>. This has the effect of adding <code>All</code> row and column labels, with corresponding values being the group statistics for all the data within a single tier:</p>

In [100]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

<p>Here, the <code>All</code> values are means without taking into account smoker versus non-smoker (the <code>All</code> columns) or any of the two levels of grouping on the rows (the <code>All</code> row).</p>

<p>To use an aggregation function other than <code>mean</code>, pass it to the <code>aggfunc</code> keyword argument. For example, <code>"count"</code> or <code>len</code> will give you a cross-tabulation (count or frequency) of group sizes (though <code>"count"</code> will exclude null values from the count within data groups, while <code>len</code> will not):</p>

In [101]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

<p>If some combinations are empty (or otherwise NA), you may wish to pass a <code>fill_value</code>:</p>

In [102]:
tips.pivot_table(index=["time", "size", "smoker"], 
                 columns="day",
                 values="tip_pct", fill_value=0)

<p>See <a href="#table_10.2">Table 10.2</a> for a summary of <code>pivot_table</code> options</p>

<table id="table_10.2">
    <caption>Table 10.2: <code>pivot_table</code> options</caption>
    <tr>
        <th>Argument</th>
        <th>Descripction</th>
    </tr>
    <tr>
        <td><code>values</code></td>
        <td>Column name or names to aggregate; by default, aggregates all numeric columns</td>
    </tr>
    <tr>
        <td><code>index</code></td>
        <td>Column names or other group keys to group on the rows of the resulting pivot table</td>
    </tr>
    <tr>
        <td><code>columns</code></td>
        <td>Column names or other group keys to group on the columns of the resulting pivot table</td>
    </tr>
    <tr>
        <td><code>aggfunc</code></td>
        <td>Aggregation function or list of functions (<code>"mean"</code> by default); can be any function valid in a <code>groupby</code> context</td>
    </tr>
    <tr>
        <td><code>fill_value</code></td>
        <td>Replace missing values in the result table</td>
    </tr>
    <tr>
        <td><code>dropna</code></td>
        <td>If <code>True</code>, do not include columns whose entries are all <code>NA</code></td>
    </tr>
    <tr>
        <td><code>margins</code></td>
        <td>Add row/column subtotals and grand total (<code>False</code> by default)</td>
    </tr>
    <tr>
        <td><code>margins_name</code></td>
        <td>Name to use for the margin row/column labels when passing <code>margins=True</code>; defaults to <code>"All"</code></td>
    </tr>
    <tr>
        <td><code>observed</code></td>
        <td>With Categorical group keys, if <code></code>True, show only the observed category values in the keys rather than all categories</td>
    </tr>
</table>    

<h3>Cross-Tabulations: Crosstab</h3>

<p>A <em>cross-tabulation</em> (or <em>crosstab</em> for short) is a special case of a pivot table that computes group frequencies. Here is an example:</p>

In [103]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

In [104]:
data

<p>As part of some survey analysis, we might want to summarize this data by nationality and handedness. You could use <code>pivot_table</code> to do this, but the <code>pandas.crosstab</code> function can be more convenient:</p>

In [105]:
pd.crosstab(data["Nationality"], data["Handedness"], 
            margins=True)

<p>The first two arguments to <code>crosstab</code> can each be an array or Series or a list of arrays. As in the tips data:</p>

In [106]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"],
            margins=True)

In [107]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS